# Running scripts in singularity

* **Difficulty level**: intermediate
* **Time need to lean**: 30 minutes or less
* **Key points**:
  * Using `container='library:xxx'` etc will enable `engine='singularity'`
  * `engine='singularity'` is explicitly needed for docker images.  

Before you run any script using singularity, please check if `singularity` is installed by checking the availability of command `singularity`. Also, it would be helpful for you to read the [sigularity user's guide](https://sylabs.io/guides/3.3/user-guide/index.html) or some online tutorial to understand how singularity works before you try to use singularity in SoS.

Although running singularity images does not need root privilege, building singularity images often requires `sudo` access. SoS provides an option `sudo=True` to the `singularity_build` action (an equivalence to command `singularity build`) to execute the command in sudo mode, but it does not accept interactive input of password. So before running any `singularity_build` action with option `sudo=True`, please run `sudo -i` to enter sudo mode, or add your username as a sudo user without password (google "sudo without password" for instructions). The latter is not safe but can be very convenient for you to prepare singularity images on a personal linux workstation.

## Running a script using singularity

SoS executes scripts with a singularity container by calling command `singularity exec` with appropriate parameters. For example, if you specify a container with a `library:` schema, sos will first pull the image, save it as a local image, and use `singularity exec` to run it:

In [1]:
%run -v1
sh: container='library://alpine:latest'
  cat /etc/os-release

HINT: Pulling image library://alpine:latest to alpine-latest.sif
NAME="Alpine Linux"
ID=alpine
VERSION_ID=3.9.2
PRETTY_NAME="Alpine Linux v3.9"
HOME_URL="https://alpinelinux.org/"
BUG_REPORT_URL="https://bugs.alpinelinux.org/"
[    1.336892] reboot: Power down


The actual `singularity exec` command executed by SoS can be shown when you execute the script in dryrun mode (with `-n` option). In this mode, SoS would print the interpolated script (if option `expand=True` is set) and the docker command to execute it:

In [2]:
%run -v1 -n
sh: container='library://alpine:latest'
  cat /etc/os-release

HINT: singularity exec  alpine-latest.sif /bin/sh /Users/bpeng1/sos/sos-docs/src/user_guide/tmpgc4nzxad/singularity_run_74690.sh
cat /etc/os-release



As you can see, the docker command looks similar to

```
singularity exec alpine-latest.sif /bin/sh /path/to/a/temp/script
```

Basically, SoS pulls the image and runs command `singularity exec` to execte the script with `/bin/sh` with the singularity image.

You can also use a docker image with singularity. However, because `docker://` images are by default executed by docker, you will need to specify the use of singularity using parameter `engine='singularity'`:

In [3]:
%run -v1
sh: container='docker://ubuntu', engine='singularity'
  cat /etc/os-release

HINT: Pulling image docker://ubuntu to ubuntu.sif
NAME="Ubuntu"
VERSION="18.04.3 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.3 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
[    4.659910] reboot: Power down


In summary, as listed [here](https://vatlab.github.io/sos-docs/doc/user_guide/sos_actions.html), you can invoke `singularity` with the following combinations of parameters `container` and `engine`:

| `container` | `engine` | execute by | example | comment | 
| -- | -- | -- | -- | -- |
| `filename.simg` | ` ` | singularity | `container='ubuntu.simg'` | |
| `shub://tag` | ` ` | singularity | `container='shub://GodloveD/lolcow'` | Image will be pulled to a local image |
| `library://tag` | ` ` | singularity | `container='library://GodloveD/lolcow'` | Image will be pulled to a local image |
| `name` | `singularity` | singularity | `container='a_dir', engine='singularity'` | treat `name` as singularity image file or directory |
| `docker://tag` | `singularity` | singularity |  `container='docker://godlovdc/lolcow', engine='singularity'`  |  |
| `file://filename` | ` ` | singularity | `container='file://ubuntu.simg'` | |

Simply put, singularity will be used by default with container `shub://`, `file://`, and `filename.simg`, but you will have to specify `engine='singularity'` if you would like to use a docker image or a directory.

### Running script with singularity image

Although singularity accepts `docker://` and `shub://` container tags, SoS always pull the image and build a local `.sif` file before executing it. If the container is used again, the local `simg` file will be used directly.

For example, the following action will pull `docker://ubuntu` and create `ubuntu.simg`

In [14]:
run: container='docker://ubuntu', engine='singularity'
  ls /

Pulling image docker://ubuntu to ubuntu.sif


ls /
Users  boot  environment  home	lib64  mnt  proc  run	singularity  sys  usr
bin    dev   etc	  lib	media  opt  root  sbin	srv	     tmp  var
[    1.372208] reboot: Power down


and you can use `container='ubuntu.sif'` directly if you have an exiting `ubuntu.sif` file

In [15]:
run: container='ubuntu.sif'
  ls /

ls /
Users  boot  environment  home	lib64  mnt  proc  run	singularity  sys  usr
bin    dev   etc	  lib	media  opt  root  sbin	srv	     tmp  var
[    1.383944] reboot: Power down


### Customize singularity image path

By default, downloaded singularity images are saved to folder `~/.sos/singularity/library`. Environment variable `SOS_SINGULARITY_LIBRARY` can be used to set customized directory, eg,

```bash
export SOS_SINGULARITY_LIBRARY=/path/to/sif_folder
```

SoS will use `*.sif` files in `$SOS_SINGULARITY_LIBRARY` if they exist. As a result you can manually place existing image files to `$SOS_SINGULARITY_LIBRARY` to avoid download attempts. 
This is useful when for example a workflow runs on a compute node without internet access.

### Binding directories (option `bind`)

A very useful feature about singularity is that you can use the container almost as a local command with access to local file system,

In [4]:
run: container='ubuntu.sif'
  wc -l ~/.bashrc

wc -l ~/.bashrc
171 /Users/bpeng1/.bashrc
[    1.347007] reboot: Power down


However, singularity only binds current working directory, `/tmp`, and your home directory. Other directories would be from within the image, or appear to be missing even if they exist on the host file system. For example, the following command lists `/usr/local` inside the image:

In [5]:
run: container='ubuntu.sif'
  ls /usr/local

ls /usr/local
bin  etc  games  include  lib  man  sbin  share  src
[    1.359987] reboot: Power down


and `/usr/local/var` would appear to be missing

In [6]:
run: container='ubuntu.sif'
  ls /usr/local/var

ls /usr/local/var
ls: cannot access '/usr/local/var': No such file or directory
[    1.367232] reboot: Power down


To allow singularity to see more directories, you can add one or more parameters to the `bind` parameter. For example, with `bind='/usr/local'`, the `singularity exec` command lists directory `/usr/local` from the host filesystem:

In [7]:
run: container='ubuntu.sif', bind='/usr/local'
  ls /usr/local

ls /usr/local
Caskroom  Frameworks  bin  include  lib      man  sbin	 texlive
Cellar	  Homebrew    etc  jamf     libexec  opt  share  var
[    1.370214] reboot: Power down


and we can see `/usr/local/var` actually exists

In [8]:
run: container='ubuntu.sif', bind='/usr/local'
  ls /usr/local/var

ls /usr/local/var
cache  db  homebrew  log  run
[    4.705033] reboot: Power down


Parameter `bind` accepts `host_dir:img_dir` pairs to mount `host_dir` from host as `img_dir` seen by the image

In [10]:
%run -v1
run: container='ubuntu.sif', bind='/usr/local/var:/myvar'
  ls /myvar

## Building singularity images

### Action `singularity_build` <a id="singularity_build"></a>

<b style='color:red'>This option does not work on mac due to perhaps a platform limitation</b>

Action `singularity_build` calls command `singularity build` with appropriate command line options. The SoS equivalence of the example in the singularity user's guide

```
singularity build lolcow.sif shub://GodloveD/lolcow
```
is the `singularity_build` action with options `src` and `dest`:

In [5]:
singularity_build(src='library://GodloveD/lolcow', dest='lolcow.sif')

FATAL:   Only remote builds are supported on this platform


ExecuteError: [0]: To reproduce this error please run 
  /usr/local/bin/singularity build lolcow.sif library://GodloveD/lolcow
from command line

As you can see from the warning message, you are recommended to build the image with root privilege. In this case, you can add option `sudo=True` when you are certain that the user is in SUDO (no password) mode:

In [ ]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow_sudo.simg', sudo=True)

You can also use action `singularity_build` to build an image from a singularity definition file. Using an example from the user's guide:

In [1]:
singularity_build: dest='lolcow.sif', sudo=True
    Bootstrap: docker
    From: ubuntu:16.04
    %post
        apt-get -y update
        apt-get -y install fortune cowsay lolcat
    %environment
        export LC_ALL=C
        export PATH=/usr/games:$PATH
    %runscript
        fortune | cowsay | lolcat 

FATAL:   Only remote builds are supported on this platform


ExecuteError: [0]: 
---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
script_2496832107755593776 in <module>
----> singularity_build('Bootstrap: docker\nFrom: ubuntu:16.04\n%post\n    apt-get -y update\n    apt-get -y install fortune cowsay lolcat\n%environment\n    export LC_ALL=C\n    export PATH=/usr/games:$PATH\n%runscript\n    fortune | cowsay | lolcat \n', dest='lolcow.sif', sudo=True)
      

FileNotFoundError: 2

Options such as `notest=True` could be add to the action. Note that the content of the definition file is indented for  clarify, but you can include the file as it is (no indentation).

## Further reading
* [Using docker with SoS](docker.html)
* [SoS actions](sos_actions.html)